In [1]:
from utils import *
from objects import *
from sweeper_helperfunctions import *

In [2]:
design = metal.designs.design_planar.DesignPlanar()
gui = metal.MetalGUI(design)
design.overwrite_enabled = True

**Defining the bounds for sweep variables with everything else fixed to be compatible with Qubit+Claw LOM sims**

In [3]:
cpw_sweep_opts = Dict(
                fillet = '49.9um',
                total_length = ["3830um","3900um","3970um"],
                trace_width = "11.7um",
                trace_gap = '5.1um')

In [4]:
claw_sweep_opts = dict(
    connection_pads=dict(
        readout = dict(connector_location = '90', 
                 connector_type = '0', 
                 claw_length = [f"{70+i*15}um" for i in range(25)],
                 ground_spacing = '4.1um', 
                 claw_gap = '5.1um', 
                 claw_width = '15um',
                 claw_cpw_width = "11.7um",
                 claw_cpw_length = '0um'
                 ),
    ),
    cross_width = '30um', # does not matter
    cross_length = '240um',  # does not matter
    cross_gap = '30um',  # does not matter
)

In [5]:
cplr_sweep_opts = Dict(prime_width = "11.7um",
            prime_gap = '5.1um',
            second_width = "11.7um",
            second_gap = '5.1um',
            coupling_space = '7.9um',
            coupling_length = ['225um', '240um', '255um'],
            open_termination = False,
            down_length = '50um')

**Update the simulation bounds to the part that you want to run on this machine**

In [6]:
sweep_opts = Dict(claw_opts = claw_sweep_opts, cpw_opts = cpw_sweep_opts, cplr_opts = cplr_sweep_opts)

# UPDATE `comp_id`!!!

In [7]:
comps_available = {
    "Sabrina":0,
    "Glinda":1,
    "Saruman":2
}

In [8]:
comp_id = "Saruman"  # Replace with actual computer name

In [9]:
N = len(comps_available)  # Number of computers
sweep_opts_chunks = chunk_sweep_options(sweep_opts, N)

In [10]:
sweep_opts_deployed = sweep_opts_chunks[comps_available[comp_id]]  # Get the i-th chunk
# sweep_opts_deployed = sweep_opts
sweep_opts_deployed

{'claw_opts': {'connection_pads': {'readout': {'connector_location': '90',
    'connector_type': '0',
    'claw_length': ['325um',
     '340um',
     '355um',
     '370um',
     '385um',
     '400um',
     '415um',
     '430um'],
    'ground_spacing': '4.1um',
    'claw_gap': '5.1um',
    'claw_width': '15um',
    'claw_cpw_width': '11.7um',
    'claw_cpw_length': '0um'}}},
 'cpw_opts': {'fillet': '49.9um',
  'total_length': ['3830um', '3900um', '3970um'],
  'trace_width': '11.7um',
  'trace_gap': '5.1um'},
 'cplr_opts': {'prime_width': '11.7um',
  'prime_gap': '5.1um',
  'second_width': '11.7um',
  'second_gap': '5.1um',
  'coupling_space': '7.9um',
  'coupling_length': ['225um', '240um', '255um'],
  'open_termination': False,
  'down_length': '50um'}}

**Creating the required projects**

In [13]:
uids = []
current_dir = os.getcwd()  # Current directory
project_path = os.path.join(current_dir, 'project_storage')  #

create_project_path(project_path)
csv_path = os.path.join(project_path, 'uids.csv')

for param in extract_QSweep_parameters(sweep_opts_deployed):
    uid = '_'.join([param["cpw_opts"]["total_length"], param["claw_opts"]["connection_pads"]["readout"]["claw_length"]])
    uid = '_'.join([uid, param["cplr_opts"]["coupling_length"]])
    uids.append(f"clt_sweep_{uid}")

write_uids_to_csv(uids, csv_path)
print(f"UIDs have been written to \n{csv_path}")

UIDs have been written to 
d:\Andre\paper_sweeps\project_storage\uids.csv


Update `csv_path` in `create_new_project.py` with the path above ^

# DELETE the Temp directory FIRST to make sure we don't run into storage issues!

# STOP AND RUN the `create_new_project.py` script within Ansys before proceeding

If all the projects are now available. Render the design into them using the following two cells

In [14]:
remove_lock_files(project_path)

Deleted: d:\Andre\paper_sweeps\project_storage\clt_sweep_3830um_325um_225um.aedt.lock
Deleted: d:\Andre\paper_sweeps\project_storage\clt_sweep_3830um_325um_240um.aedt.lock
Deleted: d:\Andre\paper_sweeps\project_storage\clt_sweep_3830um_325um_255um.aedt.lock
Deleted: d:\Andre\paper_sweeps\project_storage\clt_sweep_3830um_340um_225um.aedt.lock
Deleted: d:\Andre\paper_sweeps\project_storage\clt_sweep_3830um_340um_240um.aedt.lock
Deleted: d:\Andre\paper_sweeps\project_storage\clt_sweep_3830um_340um_255um.aedt.lock
Deleted: d:\Andre\paper_sweeps\project_storage\clt_sweep_3830um_355um_225um.aedt.lock
Deleted: d:\Andre\paper_sweeps\project_storage\clt_sweep_3830um_355um_240um.aedt.lock
Deleted: d:\Andre\paper_sweeps\project_storage\clt_sweep_3830um_355um_255um.aedt.lock
Deleted: d:\Andre\paper_sweeps\project_storage\clt_sweep_3830um_370um_225um.aedt.lock
Deleted: d:\Andre\paper_sweeps\project_storage\clt_sweep_3830um_370um_240um.aedt.lock
Deleted: d:\Andre\paper_sweeps\project_storage\clt_swe

In [18]:
CLT_epr_sweep(design, sweep_opts_deployed, project_path)

INFO 04:04PM [connect_project]: Connecting to Ansys Desktop API...
INFO 04:04PM [load_ansys_project]: 	File path to HFSS project found.
INFO 04:04PM [load_ansys_project]: 	Opened Ansys App
INFO 04:04PM [load_ansys_project]: 	Opened Ansys Desktop v2021.1.0
INFO 04:04PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    d:/Andre/paper_sweeps/project_storage/
	Project:   clt_sweep_3830um_325um_225um
INFO 04:04PM [connect_design]: No active design found (or error getting active design).
INFO 04:04PM [connect]: 	 Connected to project "clt_sweep_3830um_325um_225um". No design detected


Starting rendering...
Setting project name


INFO 04:04PM [connect_design]: 	Opened active design
	Design:    CavitySweep [Solution type: Eigenmode]
WARNING 04:04PM [connect_setup]: 	No design setup detected.
WARNING 04:04PM [connect_setup]: 	Creating eigenmode default setup.
INFO 04:04PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 04:04PM [connect_design]: 	Opened active design
	Design:    CavitySweep_hfss [Solution type: Eigenmode]
WARNING 04:04PM [connect_setup]: 	No design setup detected.
WARNING 04:04PM [connect_setup]: 	Creating eigenmode default setup.
INFO 04:04PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


Sim rendered into HFSS!


INFO 04:04PM [connect_project]: Connecting to Ansys Desktop API...
INFO 04:04PM [load_ansys_project]: 	File path to HFSS project found.
INFO 04:04PM [load_ansys_project]: 	Opened Ansys App
INFO 04:04PM [load_ansys_project]: 	Opened Ansys Desktop v2021.1.0


...

Sim rendered into HFSS!


INFO 04:23PM [connect_project]: Connecting to Ansys Desktop API...
INFO 04:23PM [load_ansys_project]: 	File path to HFSS project found.
INFO 04:23PM [load_ansys_project]: 	Opened Ansys App
INFO 04:23PM [load_ansys_project]: 	Opened Ansys Desktop v2021.1.0


Rendering is done!
Starting rendering...
Setting project name


INFO 04:23PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    d:/Andre/paper_sweeps/project_storage/
	Project:   clt_sweep_3970um_430um_255um
INFO 04:23PM [connect_design]: No active design found (or error getting active design).
INFO 04:23PM [connect]: 	 Connected to project "clt_sweep_3970um_430um_255um". No design detected
INFO 04:23PM [connect_design]: 	Opened active design
	Design:    CavitySweep [Solution type: Eigenmode]
WARNING 04:23PM [connect_setup]: 	No design setup detected.
WARNING 04:23PM [connect_setup]: 	Creating eigenmode default setup.
INFO 04:23PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 04:23PM [connect_design]: 	Opened active design
	Design:    CavitySweep_hfss [Solution type: Eigenmode]
WARNING 04:23PM [connect_setup]: 	No design setup detected.
WARNING 04:23PM [connect_setup]: 	Creating eigenmode default setup.
INFO 04:23PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


Sim rendered into HFSS!
Rendering is done!
Releasing Ansys Now


# 1. Run `correct_analysis_setup.py` to fix the Setup objects

# 2. Delete the `.lock` files again

In [19]:
remove_lock_files(project_path)

Deleted: d:\Andre\paper_sweeps\project_storage\clt_sweep_3830um_325um_225um.aedt.lock
Deleted: d:\Andre\paper_sweeps\project_storage\clt_sweep_3830um_325um_240um.aedt.lock
Deleted: d:\Andre\paper_sweeps\project_storage\clt_sweep_3830um_325um_255um.aedt.lock
Deleted: d:\Andre\paper_sweeps\project_storage\clt_sweep_3830um_340um_225um.aedt.lock
Deleted: d:\Andre\paper_sweeps\project_storage\clt_sweep_3830um_340um_240um.aedt.lock
Deleted: d:\Andre\paper_sweeps\project_storage\clt_sweep_3830um_340um_255um.aedt.lock
Deleted: d:\Andre\paper_sweeps\project_storage\clt_sweep_3830um_355um_225um.aedt.lock
Deleted: d:\Andre\paper_sweeps\project_storage\clt_sweep_3830um_355um_240um.aedt.lock
Deleted: d:\Andre\paper_sweeps\project_storage\clt_sweep_3830um_355um_255um.aedt.lock
Deleted: d:\Andre\paper_sweeps\project_storage\clt_sweep_3830um_370um_225um.aedt.lock
Deleted: d:\Andre\paper_sweeps\project_storage\clt_sweep_3830um_370um_240um.aedt.lock
Deleted: d:\Andre\paper_sweeps\project_storage\clt_swe

# 3. Start the Automated Jobs Handler by running `start_parallell_sweeps.py`

---

# Analyze the simulations

In [ ]:
import csv
import os
import pyEPR as epr


def get_pinfo(path, project_name, design_name):
    pinfo = epr.ProjectInfo(project_path=path,
                            project_name=project_name,
                            design_name=design_name)
    return pinfo

# Define the CSV file path and project directory
#csv_path = r"D:\andre\cavity_sweeps\project_storage\uids.csv" # change this
#project_path = os.path.dirname(csv_path)

# design name
design_name = "CavitySweep_hfss"

# Initialize a list to hold pinfo objects
pinfo_objects = []

# Read the project names from the CSV file

with open(csv_path, 'r') as csvfile:
    csv_reader = csv.reader(csvfile)
    header = next(csv_reader)  # Skip the header
    for row in csv_reader:
        project_name = row[0]
        print(project_name)
        # Generate the pinfo object for each project
        try:
            pinfo = get_pinfo(project_path, project_name, design_name)
            pinfo_objects.append(pinfo)
        except:
            print(f"Skipping {project_name}")


# Andre cavity sweep setup used
setup = {
        "sim_type": "epr",
        "setup": {
            "name": "Setup",
            "reuse_selected_design": True,
            "reuse_setup": True,
            "min_freq_ghz": 1,
            "n_modes": 1,
            "max_delta_f": 0.05,
            "max_passes": 50,
            "min_passes": 1,
            "min_converged": 1,
            "pct_refinement": 30,
            "basis_order": 1,
            "vars": {
                "Lj": "0nH",
                "Cj": "0fF"
            },
            "min_converged_passes": 2
        }
    }


In [ ]:
pinfo_objects[0].design_name

In [ ]:
# analayze and store the data
for i,param in enumerate(extract_QSweep_parameters(sweep_opts_deployed)):
    try:
        # read results
        pinfo = pinfo_objects[i]
        
        # setup pyEPR
        pinfo.dissipative['dielectrics_bulk'] = ['main'] # Dissipative elements: specify
        # Next 2 lines are counterinuitive, since there is no junction in this resonator.
        # However, these are necessary to make pyEPR work correctly. Please do note delete
        pinfo.design.set_variable('Lj', '12 nH')
        pinfo.design.set_variable('Cj', '1 fF')
        pinfo.validate_junction_info() # Check that valid names of variables and objects have been supplied.

        # get results
        eprh = epr.DistributedAnalysis(pinfo) # epr hfss analysis 
        epr_data = eprh.get_ansys_frequencies_all()
        freq = epr_data["Freq. (GHz)"].values[0] # GHz
        Q = epr_data["Quality Factor"].values[0] # unitless
        kappa = (freq*1e9 / Q)*1e-6 # MHz

        #store results

        data_df = {
            "design_options": {
                "coupling_type": "CLT",
                "geometry_dict": param
            },
            "sim_options": {
                "sim_type": "epr",
                "setup": setup,
            },
            "sim_results": {
                "cavity_frequency": freq,
                "Q": Q,
                "kappa": kappa
            },
            "misc": {}
        }

        # filename = f"CLT_cpw{cpw.options.total_length}_claw{claw.options.connection_pads.readout.claw_width}_clength{coupler.options.coupling_length}"
        save_simulation_data_to_json(data_df, pinfo.project_name)
    except:
        print(f"{pinfo.project_name} didn't work")